# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Villasenor  ->  J. Villasenor  |  ['J. Villasenor']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Kumar  ->  S. Kumar  |  ['S. Kumar']
J. Li  ->  J. Li  |  ['J. Li']


J. Li  ->  J. Li  |  ['J. Li']
S. Kumar  ->  S. Kumar  |  ['S. Kumar']
Arxiv has 140 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.12102


extracting tarball to tmp_2512.12102...

 done.


Retrieving document from  https://arxiv.org/e-print/2512.12304


...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.12846/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2512.13136...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.13320


extracting tarball to tmp_2512.13320...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.12102-b31b1b.svg)](https://arxiv.org/abs/2512.12102) | **Binarity at LOw Metallicity (BLOeM): Projected rotational velocities**  |
|| D. J. Lennon, et al. -- incl., <mark>J. Villasenor</mark> |
|*Appeared on*| *2025-12-16*|
|*Comments*| *16 pages, 18 figures. 3 tables are only available in electronic form. Submitted to Astronomy and Astrophysics*|
|**Abstract**|            The Binarity at LOw Metallicity (BLOeM) survey is an ESO large programme designed to obtain multi-epoch spectroscopy for 929 massive stars in the Small Magellanic Cloud (SMC). It will provide binary fractions and orbital configurations of binary systems, and search for dormant black-hole binary candidates (OB+BH). Here we present projected rotational velocities (vsini) of all sources and, using the multiplicity properties presented in previous papers, we derive the vsini distributions of apparent single stars, single lined spectroscopic (SB1) binaries, and SB2 systems. We identify a locus in the Hertzsprung-Russell diagram where rotational velocities decrease significantly; we interpret this feature as broadly corresponding to the terminal-age main sequence. The main sequence cohort is distinguished by a broad range of vsini values, but with a strong peak in the distribution in the range 30-60 km/s, close to the resolution limit of 30 km/s. Sources in this low vsini peak are distributed throughout the main sequence, and are also present in the SB1 sample, though less prominent than in the single star distribution. A preliminary analysis of the lowest vsini cohort, that includes SB1 systems, implies that roughly one third may be nitrogen rich and we speculate that this cohort is a mix of pristine single stars, long period binaries, and merger products. The SB2 systems appear to be mostly short period binaries in synchronous rotation and have vsini estimates distributed around a mean value of approximately 140 km/s . Higher vsini sources are also present in the single and SB1 systems, all of which have tail to higher vsini values, consistent with tidal and mass-transfer effects. The supergiants, with a few exceptions, have low vsini, the bulk of these systems being essentially unresolved at current spectral resolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.12304-b31b1b.svg)](https://arxiv.org/abs/2512.12304) | **Local Nonlinear Transforms effectively Reveal Primordial Information in Large-Scale Structure**  |
|| <mark>Y. Wang</mark>, H.-R. Yu, Y. Yu, P. He |
|*Appeared on*| *2025-12-16*|
|*Comments*| *5 pages, 3 figures, 1 table, comments welcomed*|
|**Abstract**|            To eliminate gravitational non-Gaussianity, we introduce the $\mathcal{Z}$-$\kappa$ transform, a simple local nonlinear transform of the matter density field that emulates the inverse of nonlinear gravitational evolution. Using $N$-body simulations, we show that the $\mathcal{Z}$-$\kappa$ transform with $\kappa=6$ or $\kappa\to\infty$ (i.e., log) substantially Gaussianizes the density distribution, and recovers the linear power spectrum. In an extended parameter space including primordial non-Gaussianity, summed neutrino mass, and $\Lambda$CDM parameters, Fisher analysis demonstrates that power spectra of transformed fields provide strong complementary constraints. A central result is that these power spectra can directly capture the local primordial non-Gaussianity imprinted in large-scale structure. This opens a new avenue for probing the physics of the early Universe with Stage-IV surveys using two-point statistics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.12353-b31b1b.svg)](https://arxiv.org/abs/2512.12353) | **Measuring the Collisional Evolution of Debris Clusters in an Asteroid System**  |
|| <mark>Y. Wu</mark>, <mark>X. Zhang</mark>, C. Huang, Y. Yu |
|*Appeared on*| *2025-12-16*|
|*Comments*| **|
|**Abstract**|            Context. Rotational instability of rubble-pile asteroids can trigger mass shedding, forming transient debris clouds that may provide the initial conditions for secondary formation in binary systems. Aims. We investigate the dynamical and collisional evolution of a debris cloud numerically generated around a Didymos-like progenitor, as a representative case for the early formation of Dimorphos. The analysis focuses on the growth and structural properties of clusters composed of centimetre- to decimetre-scale particles. Methods. We perform full-scale simulations of debris evolution around a near-critically rotating asteroid using a cross-spatial-scale approach combined with the discrete element method (DEM). To overcome computational timescale limitations, an equivalent cluster-scale simulation framework is introduced to capture the essential collisional growth processes efficiently. These simulations quantify the efficiency of cluster growth and the structural evoution within the debris cloud. Results. Our simulations reveal that particles shed from a rotationally unstable asteroid exhibit a consistent migration pattern toward low-geopotential regions, which governs the mass distribution and dynamical structure of the debris cloud. The collisional velocity are well described by a Weibull distribution (lambda = 0.0642, k = 1.8349), where low-velocity impacts favor accretion. These collisions enable clusters to grow from centimeter-decimeter scales to meter-sized bodies, developing compact, moderately porous structures (Delta I \approx 0.8, phi \approx 0.52). Collisions between meter-sized clusters do not exhibit a bouncing barrier: low-velocity impacts yield Dinkinesh-like shapes, while moderate velocities promote plastic merging and continued growth.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.12846-b31b1b.svg)](https://arxiv.org/abs/2512.12846) | **The Merian Survey: A Statistical Census of Bright Satellites of Milky Way Analogs**  |
|| Y. Pan, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-12-16*|
|*Comments*| *22 pages, 8 figures. Published in ApJ in December, 2025*|
|**Abstract**|            We present a statistical census of bright, star-forming satellite galaxies around Milky Way (MW) analogs using the first data release of the Merian Survey. Our sample consists of 393 MW analogs with stellar masses $10^{10.5} < M_{\star, \rm host} < 10^{10.9} M_\odot$ at redshifts $0.07 < z < 0.09$, all central galaxies of their own dark matter halos. Using photometric selection -- including magnitude, color, angular size, photometric redshift, and size-mass cuts -- we identify 793 satellite candidates around these 393 hosts. Our selection leverages two medium-band filters targeting H$\alpha$ and [O \textsc{iii}] emission, enabling a nearly complete sample of star-forming, Magellanic Clouds-like satellites with $M_{\star, \rm sat} \gtrsim 10^{8} M_\odot$. We find that $\sim80\%$ of hosts have 0-3 massive satellites, and $13\pm4\%$ have two satellites (similar to the MW). Satellite abundance correlates with total stellar mass, and we provide significantly improved statistics for the most massive satellites at $\log_{10}[M_{\star, \rm sat}/M_{\odot}] \gtrsim 10$. The completeness-corrected radial distribution is less centrally concentrated than an NFW profile. In contrast, the Milky Way satellites are more centrally concentrated than the 50\% richest Merian systems, but are broadly consistent with the 50\% most centrally concentrated Merian systems. Our results highlight the power of medium-band photometry for satellite identification and provide a key benchmark for studying satellite quenching, environmental effects, and hierarchical galaxy formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.13320-b31b1b.svg)](https://arxiv.org/abs/2512.13320) | **Evidence for in situ particle energization during the May-2024 event based on ASPEX instrument on board Aditya-L1**  |
|| S. Parashar, et al. -- incl., <mark>S. Kumar</mark> |
|*Appeared on*| *2025-12-16*|
|*Comments*| **|
|**Abstract**|            The interaction between interplanetary Coronal Mass Ejection (ICME) structures can alter the geo-effectiveness of the ICME events in myriad ways. Many aspects of these interaction processes are not well-understood till date. Using the energy spectra measured in two mutually orthogonal top hat analyzers (THA 1 and 2), which are part of the Solar Wind Ion Spectrometer (SWIS) subsystem of the Aditya Solar Wind Particle EXperiment (ASPEX) on board India's Aditya L1 mission, we gain insights into intricate features of ICME ICME interactions during May 2024 solar event. We report here an unprecedented two-orthogonal-plane perspective of ICME ICME interactions for the first time from the L1 point. The investigation reveals a special interaction region formed by the propagation of the forward shock driven by complex ejecta in the preceding ICME. The interaction causes the formation of a downstream region spanning over 13 hours, which propagates in the interplanetary medium. The observations reveal that this region serves as a site for proton and alpha particle energization, and the particles within this region get distributed from one plane to the other. The presence of forward shock and particle energization is confirmed by the energetic particle flux measurements by the SupraThermal and Energetic Particle Spectrometer (STEPS) of ASPEX. These observations provide an unprecedented perspective on how solar wind ions become energized and distributed in an ICME-ICME interaction region.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.13136-b31b1b.svg)](https://arxiv.org/abs/2512.13136) | **VLBI astrometry of radio stars to link radio and optical celestial reference frames III: 11 radio stars**  |
|| J. Zhang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-12-16*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            The alignment between the radio-based International Celestial Reference Frame (ICRF) and the optical Gaia Celestial Reference Frame (Gaia-CRF) is critical for multi-waveband astronomy, yet systematic offsets at the optical bright end (G<13) limit their consistency. While radio stars offer a potential link between these frames, their utility has been restricted by the scarcity of precise Very Long Baseline Interferometry (VLBI) astrometry. In this study, we present new VLBI astrometry of 11 radio stars using the Very Long Baseline Array (VLBA), expanding the existing sample with positions, parallaxes, and proper motions measured. All 11 radio stars were detected, for 10 of which parallaxes and proper motions can be estimated, reaching a precision level of <1% in the best cases. These new samples greatly contribute to the link between ICRF and Gaia-CRF at the optical bright end.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.12679-b31b1b.svg)](https://arxiv.org/abs/2512.12679) | **HEL1OS -- A Hard X-ray Spectrometer on Board Aditya-L1**  |
|| A. Nandi, et al. -- incl., <mark>S. Kumar</mark> |
|*Appeared on*| *2025-12-16*|
|*Comments*| *28 pages, 14 figures; published in the Solar Physics journal; this is the submitted version*|
|**Abstract**|            HEL1OS (High Energy L1 Orbiting X-ray Spectrometer) is one of the remote sensing payloads on board Aditya-L1 mission designed to continuously monitor and measure the time-resolved spectra of solar flares between 8 keV and 150 keV. This broad energy range has been covered by using compound semiconductor detectors: cadmium telluride (CdTe: 8 - 70 keV) and cadmium zinc telluride (CZT: 20 - 150 keV) with geometric areas of 0.5 cm$^2$ and 32 cm$^2$, respectively. A stainless steel collimator provides a field-of-view of 6$^\circ$ $\times$ 6$^\circ$ optimized to limit the off-axis response while keeping the design within the instrument mass constraints. The in-house designed low-noise digital pulse processing-based front-end electronics has achieved a spectral resolution of $\approx$ 1 keV at 14 keV (CdTe) and $\approx$ 7 keV at 60 keV (CZT). The instrument is also equipped with processing and power electronics to process the signal, drive the electronics, bias the detectors with required low and high voltages for optimal performance of the overall system. In this article, we present design aspects of the instrument, results from the pre-launch ground-based tests, and the in-orbit operations, which have indicated optimal performance in line with that expected.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.12102/./figures/plot_hrd_vsini_color.png', 'tmp_2512.12102/./figures/trans_vsini_drop.png', 'tmp_2512.12102/./figures/plot_hist_binaries.png']
copying  tmp_2512.12102/./figures/plot_hrd_vsini_color.png to _build/html/
copying  tmp_2512.12102/./figures/trans_vsini_drop.png to _build/html/
copying  tmp_2512.12102/./figures/plot_hist_binaries.png to _build/html/
exported in  _build/html/2512.12102.md
    + _build/html/tmp_2512.12102/./figures/plot_hrd_vsini_color.png
    + _build/html/tmp_2512.12102/./figures/trans_vsini_drop.png
    + _build/html/tmp_2512.12102/./figures/plot_hist_binaries.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{{km s}^{-1}}$
$\newcommand{\teff}{T_\mathrm{eff} }$
$\newcommand{\logg}{\log g }$
$\newcommand{\vt}{\varv_{\rm{t}}}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Rsun}{R_\odot}$
$\newcommand{\Tsun}{T_\odot}$
$\newcommand{\vsini}{\varv\sin i}$
$\newcommand{\vsinift}{\varv\sin i_{\rm FT}}$
$\newcommand{\vsinifwhm}{\varv\sin i_{\rm FWHM}}$
$\newcommand{\ve}{\varv_{\rm{esc}}}$
$\newcommand{\vcrit}{\varv_{\rm{crit}}}$</div>



<div id="title">

# Binarity at LOw Metallicity (BLOeM)$\thanks{Based on observations collected at the European Southern Observatory under ESO program ID 112.25R7}$  : Projected rotational velocities

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.12102-b31b1b.svg)](https://arxiv.org/abs/2512.12102)<mark>Appeared on: 2025-12-16</mark> -  _16 pages, 18 figures. 3 tables are only available in electronic form. Submitted to Astronomy and Astrophysics_

</div>
<div id="authors">

D. J. Lennon, et al.

</div>
<div id="abstract">

**Abstract:** The Binarity at LOw Metallicity (BLOeM) survey is an ESO large programme designed to obtain multi-epoch spectroscopy for 929 massive stars in the Small Magellanic Cloud (SMC).It will provide binary fractions and orbital configurations of binary systems, and search for dormant black-hole binary candidates (OB+BH).Here we present projected rotational velocities ( $\vsini$ ) of all sources and, using the multiplicity properties presented in previous papers, we derive the $\vsini$ distributions of apparent single stars, single lined spectroscopic (SB1) binaries, and SB2 systems.We identify a locus in the Hertzsprung-Russell diagram where rotational velocities decrease significantly; we interpret this feature as broadly corresponding to the terminal-age main sequence.The main sequence cohort is distinguished by a broad range of $\vsini$ values, but with a strong peak in the distribution in the range 30--60 $\kms$ , close to the resolution limit of $\sim$ 30 $\kms$ .Sources in this low $\vsini$ peak are distributed throughout the main sequence, and are also present in the SB1 sample, though less prominent than in the single star distribution.A preliminary analysis of the lowest $\vsini$ cohort, that includes SB1 systems, implies that roughly one third may be nitrogen rich and we speculate that this cohort is a mix of pristine single stars, long period binaries, and merger products.The SB2 systems appear to be mostly short period binaries in synchronous rotation and have $\vsini$ estimates distributed around a mean value of $\approx$ 140 $\kms$ .Higher $\vsini$ sources are also present in the single and SB1 systems, all of which have tail to higher $\vsini$ values, consistent with tidal and mass-transfer effects.The supergiants, with a few exceptions, have low $\vsini$ , the bulk of these systems being essentially unresolved at current spectral resolution ( $\sim$ 30 $\kms$ ).

</div>

<div id="div_fig1">

<img src="tmp_2512.12102/./figures/plot_hrd_vsini_color.png" alt="Fig17" width="100%"/>

**Figure 17. -** HRD of the BLOeM OB sources colour-coded according to the ratio $\vsini$/$\vcrit$ as indicated in the inset. Stellar parameters are taken from [Bestenlehner, et. al (2025)](https://ui.adsabs.harvard.edu/abs/2025MNRAS.540.3523B) and the evolutionary tracks, for illustration, are those of [Schootemeijer, et. al (2019)](https://ui.adsabs.harvard.edu/abs/2019A&A...625A.132S) assuming $\alpha_{\rm ov}=0.33$ and $\alpha_{\rm sc}=10$. Tracks are labeled with their initial mass, in solar units. The large trapezium bounded by diagonal dashed lines between luminosities 4.2 and 5.5 indicates the approximate extent of the fast rotators and main sequence, as discussed in section \ref{demographics}. (*fig:hrd*)

</div>
<div id="div_fig2">

<img src="tmp_2512.12102/./figures/trans_vsini_drop.png" alt="Fig4" width="100%"/>

**Figure 4. -** Distribution of $\vsini$ as a function of distance from the ZAMS as indicated by $\teff$ as defined by the lower and upper luminosity boundaries illustrated in Fig. \ref{fig:hrd}.
    The vertical dashed line indicates the TAMS at these luminosities as defined by the approximate position of the decrease in the $\vsini$ distribution. (*fig:vsinidrop*)

</div>
<div id="div_fig3">

<img src="tmp_2512.12102/./figures/plot_hist_binaries.png" alt="Fig5" width="100%"/>

**Figure 5. -** The $\vsini$ distributions of non-OBe main sequence (MS) systems together with single, SB1 and SB2 systems. Sample sizes as indicated by the insets. In the SB2 panel, lower right, the $\vsini$ distribution of the eclipsing systems (ECL) in BLOeM is also shown (dashed line). The histogram bin size is 20 $\kms$. The total of main-sequence sources is greater than the sum of single, SB1 and SB2 as some main-sequence sources have unclear multiplicity designations, as discussed at the beginning of section \ref{results}.  (*fig:hist_binaries*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.12102"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
